# Naive Bayes Spam Filter
This project consists of using a multinomial Naive Bayes algorithm in order to classify incoming SMS messages as spam or non-spam (ham). To train our Naive Bayes model, we will be using a dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) consisting of 5,572 SMS messages that have already been classified by humans.

## Data Exploration

In [1]:
import pandas as pd

smsSpam = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                      names=['Label', 'SMS'])

smsSpam.shape

(5572, 2)

In [2]:
smsSpam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
smsSpam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

A quick exploration of the dataset shows that each SMS is labelled as spam or ham. Approximately 87% of the messages are labelled as ham, with the remaining 13% labelled as spam. This is sample is representative since most messages are not expected to be spam. We should be wary of class imbalance issues during training however.

## Splitting Training and Testing Data
The labelled dataset provided will be split into a training set (80%), and a testing set (20%). The testing set will be used to evaluate the performance of our model in an unbiased manner.

In [4]:
# Randomize the entire dataset
smsShuffled = smsSpam.sample(frac=1, random_state=1)

# Split shuffled data into a training and testing set
trainFrac = 0.8
splitIndex = round(smsShuffled.shape[0] * trainFrac)
train = smsShuffled[:splitIndex].reset_index(drop=True)
test = smsShuffled[splitIndex:].reset_index(drop=True)

In [5]:
train.shape

(4458, 2)

In [6]:
train['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [7]:
train.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
test.shape

(1114, 2)

In [9]:
test['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

In [10]:
test.head()

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


The training and testing splits look good since a similar class representation is maintained (stratified split: 87% ham, 13% spam)

## Data Cleaning
First, I clean the text data by removing all non-alphanumeric characters, and converting the text to lower case.

In [11]:
# Remove non-alphanumeric characters
train['SMS'] = train['SMS'].str.replace(r'\W', ' ')

# To Lower case
train['SMS'] = train['SMS'].str.lower()

train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating a Vocabulary
The Naive Bayes algorithm requires a unique set of words that occur to form a vocabulary. First we will create a list of all unique words that are present in the training set.

In [12]:
## Create list of all unique words in train set
tokenized = train['SMS'].str.split()
vocabulary = []
for sms in tokenized:
    for word in sms:
        if word not in vocabulary:
            vocabulary.append(word)
            

In [13]:
len(vocabulary)

7783

There are 7782 unique words in the training set. Now we will format the counts of each word for each SMS in the form of a table, so that they can be easily referenced by our algorithm. These term frequencies will act as the feature inputs to our prediction model.

In [14]:
# Initialize word count dictionary
word_counts_per_sms = {unique_word: [0] * len(tokenized) for unique_word in vocabulary}

# Count word occurances
# Index to track sms number
for index, sms in enumerate(tokenized):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        

In [15]:
word_count_matrix = pd.DataFrame(word_counts_per_sms)
word_count_matrix.head()

,yep,by,the,pretty,sculpture,yes,princess,are,you,going,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Group the cleaned training data
train_clean = pd.concat([train['Label'], tokenized, word_count_matrix], axis=1)
train_clean.head()

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants
Before beginning to construct our Naive Bayes algorithm, we will calculate some useful constants that we will be using:

- $P(Spam)$ : probability of the message being spam
- $P(Ham)$ : probability of the message being ham
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$ : total number of words in spam / ham / vocabulary

Note that we will be using Laplace smoothing, which uses a constant parameter of 1. This is done to deal with cases where the trained model encounters words that have not appeared during training.

In [17]:
train_spam_ham_prop = train_clean['Label'].value_counts(normalize=True)
train_spam_ham_prop

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [18]:
# Separate Spam and Ham in train set
train_spam = train_clean[train_clean['Label']=='spam']
train_ham = train_clean[train_clean['Label']=='ham']

# Constant Probabilities
p_spam = train_spam_ham_prop['spam']
p_ham = train_spam_ham_prop['ham']

# Constant Counts - Total # of words in spam / ham / vocabulary
n_spam = train_spam.sum(numeric_only=True).sum()
n_ham = train_ham.sum(numeric_only=True).sum()
n_vocabulary = len(vocabulary)

# Smoothing Parameter
alpha = 1

print(p_spam, p_ham, n_spam, n_ham, n_vocabulary)

0.13458950201884254 0.8654104979811574 15190 57237 7783


## Calculating Parameters
Next, we will calculate the following conditional probabilities for each unique word $w_i$ using the training data.

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}


\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [19]:
# Initialize dictionaries to store cond. Prob. for Spam | Ham
p_wi_given_spam = {}
p_wi_given_ham = {}

for word in vocabulary:
    p_wi_given_spam[word] = 0
    p_wi_given_ham[word] = 0


In [20]:
# Calculate conditional Probs for each word
for word in vocabulary:
    n_wi_given_spam = train_spam[word].sum()
    p_wi_given_spam[word] = (n_wi_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    
    n_wi_given_ham = train_ham[word].sum()
    p_wi_given_ham[word] = (n_wi_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    

# Message Classification
With all the parameters known, we can create the spam filter. The algorithm will do the following:

- Take in an input message containing words $(w_1, ..., w_n)$
- Calculate $P(Spam | w_1, ..., w_n)$ and $P(Ham | w_1, ..., w_n)$
where 
\begin{equation}
P(Spam | w_1,..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) 
\end{equation}

\begin{equation}
P(Ham | w_1,..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}
- Compare $P(Spam | w_1, ..., w_n)$ and $P(Ham | w_1, ..., w_n)$ to see which probability is larger, and classify accordingly.

In [21]:
import re

# Naive Bayes Classification function
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Calculate Conditional Prob. given a new message
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_wi_given_spam:
            p_spam_given_message *= p_wi_given_spam[word]
        if word in p_wi_given_ham:
            p_ham_given_message *= p_wi_given_ham[word]  

    # print('P(Spam|message):', p_spam_given_message)
    # print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'Equal probabilities, requires human classification'

In [22]:
# Test function
spam_example = 'WINNER!! This is the secret code to unlock the money: C3421.'
ham_example = 'Sounds good, Tom, then see u there'

print(classify(spam_example))
print(classify(ham_example))

spam
ham


## Spam Filter Accuracy
To test the performance of the algorithm, we will apply it to the testing set and evaluate its accuracy.

In [23]:
test['prediction'] = test['SMS'].apply(classify)
test.head()

,Label,SMS,prediction
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [24]:
# Measure accuracy
correct = sum(test['Label'] == test['prediction'])
total = test.shape[0]

accuracy = correct/total
print('test accuracy = ', accuracy)

test accuracy =  0.9874326750448833


In [25]:
from sklearn.metrics import classification_report

classReport = classification_report(y_true = test['Label'], y_pred = test['prediction'], 
                                    labels=['ham','spam'] )
print(classReport)

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       967
        spam       0.97      0.95      0.96       147

   micro avg       0.99      0.99      0.99      1114
   macro avg       0.98      0.97      0.97      1114
weighted avg       0.99      0.99      0.99      1114



The Naive Bayes spam filter achieves an accuracy of 98.7% on the test set. In addition, the weighted average of precision and recall of the algorithm are both 99% which is great.